# Intro

In this section, there are many examples, most of which are run in Docker. This page shows typical tricks used to run and manipulate SQL servers from a Docker container.

## Running servers

First, you have to run a container with the SQL server.

### Postgres

A key feature of running a PostgreSQL container is that you must pass the `POSTGRES_PASSWORD` environment variable. Find out more on the [PostgreSQL page on Docker Hub](https://hub.docker.com/_/postgres).

---

After running the following cell, you'll have a Docker container named `intro_example_pg`.

In [5]:
docker run --rm -d\
    -e POSTGRES_PASSWORD=postgres\
    --name intro_example_pg\
    postgres:15.4 &> /dev/null

**Note**: Don't forget to stop the containers afterward.

In [6]:
docker stop intro_example_pg

intro_example_pg


### Clickhouse

Creating a ClickHouse server is really straightforward.

---

The following cell creates a new Docker container named `intro_example_click`.

In [7]:
docker run -d --rm\
    --name intro_example_click\
    clickhouse/clickhouse-server:24

439b3453277216d74e009371674e05f09945a2bbf1f84efb7f12cadd7b4e4420


**Note**: Don't forget to stop the container afterward.

In [8]:
docker stop intro_example_click

intro_example_click


## Initialisation scripts

If you need to organise the initialisation at container startup, you can mount a folder containing `*.sql` and `*.sh` initialisation scripts on `/docker-entrypoint-initdb.d`. This will execute the files in `/docker-entrypoint-initdb.d`.

---

In the following example, I start the docker container with the folder mounted as `/docker-entrypoint-initdb.d`. This folder only contains a [create_table.sql](build_container/initialisation_scripts/create_table.sql) which creates an empty `main_table`. Then I check the existence of the table that should be created by 
`create_table.sql`.

In [1]:
docker run --rm -d\
    -v ./build_container/initialisation_scripts:/docker-entrypoint-initdb.d\
    -e POSTGRES_PASSWORD=docker_app \
    --name init_scripts_example \
    postgres:15.4 &> /dev/null

sleep 5
echo "=====CHECK TABLE====="
docker exec init_scripts_example bash -c "psql -U postgres -d postgres -c \"\dt\""

docker stop  init_scripts_example &> /dev/null

=====CHECK TABLE=====
           List of relations
 Schema |    Name    | Type  |  Owner   
--------+------------+-------+----------
 public | main_table | table | postgres
(1 row)

